In [17]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec

import numpy as np
import pandas as pd
import os
import shutil


from numba import jit

colors = [
'black','grey','brown','slategrey','darkviolet','darkmagenta',
'blue','blue','blue',
'red','red','red',
'green','green','green',
'cyan','cyan','cyan',
'orange','orange','orange',
    
'orange','firebrick','lawngreen','dodgerblue','crimson','orchid','slateblue''darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink']


Autosaving every 180 seconds


In [18]:
import scipy
import scipy.spatial

@jit
def find_spike_match(s1, s2):
    max_dist = 15
    ctr=0
    for s in s1:
        dist= np.min(np.abs(s-s2))
        if dist<=max_dist:
            ctr+=1
    
    return (ctr)

def find_template_match(data_in, 
                        templates_gt,
                        spike_train_yass,
                        spike_train_gt):
      
    template=data_in[0]
    unit = data_in[1]
    print ("UNIT: ", unit)
        
    template = template.T.ravel()
#     plt.plot(template)
#     plt.show()
#     return (_,_,_,_)

    best_result = 0
    best_roll=0 
    best_match = None
    for t in range(templates_gt.shape[0]):
        data2 = templates_gt[t]
        for k in range(-50,50,1):
            data_test = np.roll(data2,k,axis=0).T.ravel()
            result = 1-scipy.spatial.distance.cosine(template, data_test)
                
            if result>best_result:
                best_roll = k
                best_result = result
                best_match = t

    # compute # matches
    idx_yass = np.where(spike_train_yass[:,1]==unit)[0]
    #print ("spike_train_yass units: ", spike_train_yass[:,1])
    times_yass = spike_train_yass[idx_yass,0]
    #print ("times yass: ", times_yass[:20])

    # find match times
    idx_match = np.where(spike_train_gt[:,1]==best_match+1)[0]
    times_match = spike_train_gt[idx_match,0] + 24
    #print ("times match: ", times_match[:20])
    
    # find matches
    tot_matches = find_spike_match(times_yass, times_match)

    # plot figures
    if False:
        plt.plot(template)
        plt.plot(templates_gt[best_match].T.ravel())
        plt.show()
        
    # compute purity, 
    tot_sorted = idx_yass.shape[0]
    tot_ground_truth = times_match.shape[0]
    n_fn = max((tot_ground_truth-tot_matches), 0)
    n_fp = max((tot_sorted-tot_matches),0)
    
    purity =  tot_matches/ (tot_matches+n_fp)
    accuracy = tot_matches / (n_fn+tot_matches+n_fp)
    
    return (best_match, best_result, best_roll, purity, accuracy,
           n_fn, n_fp, tot_matches, tot_sorted, tot_ground_truth)


def find_match_spikes(unit, spike_train_yass, spike_train_match):
    
    # compute # matches
    idx_yass = np.where(spike_train_yass[:,1]==unit)[0]
    times_yass = spike_train_yass[idx_yass,0]
    #print ("spike_train_yass units ", unit, times_yass.shape) 
    #if unit!=190:
    #    return (_, _, _, _, _, _, _, _)
    #print ("times yass: ", times_yass[:20])

    best_match_test = 0
    accuracy_test = 0
    for best_match in np.unique(spike_train_gt[:,1]):
        # find match times
        idx_match = np.where(spike_train_gt[:,1]==best_match)[0]
        times_match = spike_train_gt[idx_match,0] + 24
        #print ("times match: ", times_match[:20])

        if times_match.shape[0]==0:
            continue
        # find matches
        tot_matches = find_spike_match(times_yass, times_match)

        # compute purity, 
        tot_sorted = idx_yass.shape[0]
        tot_ground_truth = times_match.shape[0]
        n_fn = max((tot_ground_truth-tot_matches), 0)
        n_fp = max((tot_sorted-tot_matches),0)

        purity =  tot_matches/ max((tot_matches+n_fp),1)
        accuracy = tot_matches / max((n_fn+tot_matches+n_fp),1)

        if accuracy>accuracy_test:
            accuracy_test = accuracy
            best_match_test = best_match
            #print ("accuracy_test: ", accuracy_test)
            #print ("best_match_test: ", best_match_test)
    
    # 
    idx_match = np.where(spike_train_gt[:,1]==best_match_test)[0]
    times_match = spike_train_gt[idx_match,0] + 24

    # find matches
    tot_matches = find_spike_match(times_yass, times_match)

    # compute metrics, 
    tot_sorted = times_yass.shape[0]
    tot_ground_truth = times_match.shape[0]
    
    n_fn = max((tot_ground_truth-tot_matches), 0)
    n_fp = max((tot_sorted-tot_matches),0)

    purity =  tot_matches/ max((tot_matches+n_fp),1)
    accuracy = tot_matches / max((n_fn+tot_matches+n_fp),1)
    print ("unit: ", unit, ", accuracy: ", accuracy)
    
#     print (best_match_test, purity, accuracy,
#            n_fn, n_fp, tot_matches, tot_sorted, tot_ground_truth)
    
    return (best_match_test, purity, accuracy,
           n_fn, n_fp, tot_matches, tot_sorted, tot_ground_truth)


def plot_results(templates_yass, spike_train_yass, spike_train_gt, 
                 results,
                idx_in):
    
    print (templates_yass.shape)
    ptps_yass = templates_yass.ptp(1).max(1)
    print ("len results: ", len(results))

    purity = []
    sizes= []
    accuracy = []
    fp = []
    fn = []
    for k in range(len(results)):
        best_match_test = results[k][0]
        purity.append(results[k][1])
        tot_sorted = results[k][6]
        tot_ground_truth = results[k][7]
        tot_matches = results[k][5]

        #fp.append(results[k][3]/(tot_ground_truth)
        fp.append((tot_matches/tot_sorted))
        fn.append((tot_matches)/(tot_ground_truth))

        accuracy_local = results[k][5]/(results[k][5]+results[k][3]+results[k][4]) 
        accuracy.append(accuracy_local)

        idx = np.where(spike_train_yass[:,1]==k)[0]
        sizes.append(idx.shape[0])

        if ptps_yass[k]>28:
            print ("yass: ", k, ", gt: ", best_match_test, " ptp: ", 
                   ptps_yass[k], " accuracy: ", accuracy_local)

    sizes=np.array(sizes)
    ptps = templates_yass.ptp(1).max(1)[idx_in]

    ax = plt.subplot(2,2,1)
    print (len(ptps), len(purity))
    plt.scatter(ptps, purity, s=sizes/20.,c='blue',alpha=.5)
    text = '\ntot_matches / (tot_maches + fp)'
    plt.ylabel("Purity / Precision (flatiron): "+text,fontsize=10)
    plt.xlabel("PTP (SU)",fontsize=10)
    plt.ylim(0,1)
    plt.xlim(left=0)
    
    ax=plt.subplot(2,2,2)
    plt.scatter(ptps, accuracy, s=sizes/20.,c='red',alpha=.5)
    plt.xlabel("PTP (SU)",fontsize=10)
    text = '\ntot_matches / (tot_maches + fp + fn)'
    plt.ylabel("Accuracy: "+text,fontsize=10)
    plt.ylim(0,1)
    plt.xlim(left=0)

    ax=plt.subplot(2,2,3)
    plt.scatter(ptps, fp, s=sizes/20.,c='orange',alpha=.5)
    plt.xlabel("PTP (SU)",fontsize=10)
    text = '\n(spikes_sorted-tot_matches)/spikes_sorted'
    plt.ylabel("FP: "+text,fontsize=10)
    plt.ylim(0,1)
    plt.xlim(left=0)

    ax=plt.subplot(2,2,4)
    plt.scatter(ptps, fn, s=sizes/20.,c='green',alpha=.5)
    plt.xlabel("PTP (SU)",fontsize=10)
    text = '\n(ground_truth - tot_matches)/ground_truth'
    plt.ylabel("FN: "+text,fontsize=10)
    plt.ylim(0,1)
    plt.xlim(left=0)

    plt.suptitle("Match algorithm: accuracy metric (no cosine similarity)")
    #ax.tick_params(axis='both', which='major', labelsize=30)
    plt.show()
    # visualize recomputed templates over time;
    
def binary_reader_waveforms(filename, n_channels, n_times, spikes, data_type='float32'):
    ''' Reader for loading raw binaries
    
        standardized_filename:  name of file contianing the raw binary
        n_channels:  number of channels in the raw binary recording 
        n_times:  length of waveform 
        spikes: 1D array containing spike times in sample rate of raw data
        channels: load specific channels only
        data_type: float32 for standardized data
    
    '''

    # ***** LOAD RAW RECORDING *****
    wfs=[]
    if data_type =='float32':
        data_len = 4
    else:
        data_len = 2
        
    with open(filename, "rb") as fin:
        for ctr,s in enumerate(spikes):
                #print (ctr,s)
            # index into binary file: time steps * 4  4byte floats * n_channels
            try:
                fin.seek(s * data_len * n_channels, os.SEEK_SET)
                wfs.append(np.fromfile(
                    fin,
                    dtype=data_type,
                    count=(n_times * n_channels)).reshape(n_times, n_channels))

            except:
                pass
    fin.close()
    return np.array(wfs)

In [39]:
# load ground truth data
spike_train_gt = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/ground_truth/spike_train.npy')
templates_gt = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/ground_truth/templates.npy')
print (templates_gt.shape)
ptps_gt = templates_gt.ptp(1).max(1)

# load sorted data
templates_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output/templates.npy')
print (templates_yass.shape)
ptps_yass = templates_yass.ptp(1).max(1)
spike_train_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output/spike_train.npy')

import parmap
# KS2 vs. Yass
if False:
    results = parmap.map(find_template_match, 
                     list(zip(templates_yass,np.arange(templates_yass.shape[0]))),
                     templates_gt,
                     spike_train_yass,
                     spike_train_gt,
                     pm_processes=12)
else:
    if False:
        for unit in range(10):
            find_match_spikes(unit, 
                 spike_train_yass,
                 spike_train_gt)

    else:
        results = parmap.map(find_match_spikes, #(unit, spike_train_yass, spike_train_match):
                 np.arange(templates_yass.shape[0]), 
                 spike_train_yass,
                 spike_train_gt,
                 pm_processes=12)

        
print ("DONE")

np.save('/media/cat/4TBSSD/data/spikingforest/results.npy', results)

(708, 101, 60)
(217, 101, 60)
accuracy_test:  0.0026879044300647087
best_match_test:  0accuracy_test: 
 accuracy_test: 0.0021697203471552555 
accuracy_test: accuracy_test: 0.003830243603493182
 best_match_test: accuracy_test: best_match_test:  0.0037491513745199568  
 0.00517512244709361500.0102019071972723891best_match_test: 



 best_match_test: accuracy_test:  best_match_test: 0
 0 accuracy_test: 0.003912363067292645 0

0.0015186028853454822
best_match_test: 
best_match_test:  3 accuracy_test: 0
 
accuracy_test: 0.0072699409434103315accuracy_test: accuracy_test:  
accuracy_test:   0.004342790966994789best_match_test: 
0.004333333333333333  0.0099823840281855550.004108182129407737best_match_test: 1 



1best_match_test: best_match_test: 
best_match_test:    11110

accuracy_test: accuracy_test: 
 0.005111167901865576 accuracy_test: 
best_match_test: 0.0020326733418655424
accuracy_test:   accuracy_test: 0.004208071385114582 5best_match_test:   
0.0057003257328990230.001638001638001638


best_match_test:  0
accuracy_test:  0.002717391304347826
best_match_test:  2
accuracy_test:  0.004059301094246382
unit:  best_match_test:  584 , accuracy: 
 0.9824868651488616
accuracy_test:  0.005429071803852889
accuracy_test: unit:  best_match_test:   190.0007884880741178799
 
, accuracy: best_match_test:  accuracy_test:   00.416135881104034
0.03653846153846154

accuracy_test: best_match_test:   0.0008724100327153762401
best_match_test: 
 accuracy_test:  10.002577669244335911

accuracy_test: best_match_test:   0.00166354751507589930
best_match_test: 
accuracy_test:  accuracy_test:  20.005558450581900296 

0.003942551393973529best_match_test: accuracy_test: 
 best_match_test:   0.00189349112426035539
2best_match_test: 

 4
accuracy_test:  0.004154447702834799
best_match_test:  7accuracy_test:  
0.004967855055523086
best_match_test:  18accuracy_test: 
 0.004335573379579449
accuracy_test: best_match_test:   0.7802607076350093accuracy_test: 32

 best_match_test:  0.006054490413723511461


Process ForkPoolWorker-190:


0.0025929127052722557

Process ForkPoolWorker-192:
Process ForkPoolWorker-189:
Process ForkPoolWorker-183:
Process ForkPoolWorker-188:
Process ForkPoolWorker-185:
Traceback (most recent call last):
Process ForkPoolWorker-182:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/cat/anaconda3/envs/spikingforest/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/cat/anaconda3/envs/spikingforest/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Process ForkPoolWorker-191:
  File "/home/cat/anaconda3/envs/spikingforest/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/cat/anaconda3/envs/spikingforest/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/cat/anaconda3/envs/spikingforest/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*s

KeyboardInterrupt: 

In [3]:
# reload old or new data
if True:
    results = np.load('/media/cat/4TBSSD/data/spikingforest/results_original.npy', allow_pickle=True)
    # load sorted data
    templates_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output_original/templates.npy', allow_pickle=True)
    print (templates_yass.shape)
    ptps_yass = templates_yass.ptp(1).max(1)
    spike_train_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output_original/spike_train.npy')
    
else:
    results = np.load('/media/cat/4TBSSD/data/spikingforest/results_old.npy', allow_pickle=True)
    # load sorted data
    templates_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output_old/templates.npy')
    print (templates_yass.shape)
    ptps_yass = templates_yass.ptp(1).max(1)
    spike_train_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output_old/spike_train.npy')


(221, 101, 60)


In [35]:
# Burst matching function;
templates_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output_original/templates.npy', allow_pickle=True)
print (templates_yass.shape)
spike_train_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output_original/spike_train.npy')

# ground truth data

spike_train_gt = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/ground_truth/spike_train.npy')
templates_gt = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/ground_truth/templates.npy')

# load original results
results_original = np.load('/media/cat/4TBSSD/data/spikingforest/results_original.npy',allow_pickle=True)

spike_train_yass_merged = spike_train_yass.copy()
merges = []
ctr2=0
for unit1 in range(templates_yass.shape[0]):
    for unit2 in range(templates_yass.shape[0]):
        if unit1==unit2:
            continue
        if unit1>unit2:
            continue

        temp1 = templates_yass[unit1]
        temp2 = templates_yass[unit2]

        temp1_norm = -temp1/np.min(temp1)
        temp2_norm = -temp2/np.min(temp2)

        diff = temp1_norm - temp2_norm
        if np.max(np.abs(diff))<=0.190:
            print (unit1, unit2, " diff: ", np.max(np.abs(diff)))
            #ax=plt.subplot(5,6,ctr2+1)
            merges.append(np.sort([unit1, unit2]))
            temp1 = templates_yass[unit1]
            temp2 = templates_yass[unit2]
            
            # 
            temp1 = -temp1/np.min(temp1)
            temp2 = -temp2/np.min(temp2)
            #plt.plot(temp1-temp2, c='green',alpha=.5)
            #plt.plot(temp2, c='red', alpha=.5)
            #plt.ylim(-0.1,0.1)

            idx = np.where(spike_train_yass_merged[:,1]==unit2)[0]
            spike_train_yass_merged[idx,1]=unit1
            idx2 = np.where(spike_train_yass_merged[:,1]==unit2)[0]
            idx3 = np.where(spike_train_yass_merged[:,1]==unit1)[0]
            print ("units: ", unit1, unit2, " match", " ", idx.shape[0], idx2.shape[0], idx3.shape[0],
                   ", accuracy1: ", round(results_original[unit1][2],3), 
                   ", accuracy2: ", round(results_original[unit2][2],3),
                   ", total: ", round(results_original[unit1][2] + results_original[unit2][2],3))
            
            ctr2+=1
plt.show()

# reorder dta
spike_train_final = np.zeros((0,2),'int32')
ctr=1
for k in np.unique(spike_train_yass_merged[:,1]):
    idx = np.where(spike_train_yass_merged[:,1]==k)[0]
    
    temp = np.zeros((idx.shape[0],2), 'int32')
    
    temp[:,0] = spike_train_yass_merged[idx,0]
    temp[:,1] = ctr
    spike_train_final = np.vstack((spike_train_final, temp))
    
    ctr+=1

print (spike_train_final.shape)
spike_train_yass_merged = spike_train_final.copy()
print (spike_train_yass_merged)

spike_train_yass_merged[:,1]-=1

# load sorted data
np.save('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output/spike_train_merged.npy',
       spike_train_yass_merged)




(221, 101, 60)
4 207  diff:  0.1671235
units:  4 207  match   8192 0 10997 , accuracy1:  0.318 , accuracy2:  0.254 , total:  0.572
7 8  diff:  0.14306289
units:  7 8  match   441 0 1140 , accuracy1:  0.31 , accuracy2:  0.314 , total:  0.624
15 35  diff:  0.09507002
units:  15 35  match   2975 0 5883 , accuracy1:  0.38 , accuracy2:  0.421 , total:  0.8
18 169  diff:  0.11288212
units:  18 169  match   2832 0 5438 , accuracy1:  0.437 , accuracy2:  0.503 , total:  0.939
20 180  diff:  0.10584998
units:  20 180  match   780 0 1510 , accuracy1:  0.398 , accuracy2:  0.476 , total:  0.874
24 42  diff:  0.09819445
units:  24 42  match   2870 0 5345 , accuracy1:  0.416 , accuracy2:  0.519 , total:  0.935
44 171  diff:  0.08722196
units:  44 171  match   2972 0 5508 , accuracy1:  0.387 , accuracy2:  0.518 , total:  0.905
47 124  diff:  0.1345562
units:  47 124  match   2950 0 5848 , accuracy1:  0.335 , accuracy2:  0.417 , total:  0.753
52 199  diff:  0.08053971
units:  52 199  match   2873 0 549

In [36]:
# fix templates
templates_del = []
for k in merges:
    templates_del.append(np.sort(k)[1])
templates_del = np.unique(templates_del)
templates_merged = np.delete(templates_yass,templates_del,axis=0)
print ("tempaltes merged: ", templates_merged.shape)

np.save('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output/templates_merged.npy',
       templates_merged)



tempaltes merged:  (197, 101, 60)


In [37]:
import parmap
print (templates_merged.shape)
ptps = templates_merged.ptp(1).max(1)
idx = np.argsort(ptps)[::-1]

idx_in = idx

idx_in = np.arange(templates_merged.shape[0])
print ("testing units: ", idx_in, ptps[idx_in])

results_merged = parmap.map(find_match_spikes, 
                            #np.arange(templates_merged.shape[0]), 
                            idx_in, 
                            spike_train_yass_merged,
#                             np.arange(templates_yass.shape[0]),
#                             spike_train_yass,
                            spike_train_gt,
                            pm_processes=12)
print ("DONE")
np.save('/media/cat/4TBSSD/data/spikingforest/results_merged.npy', results_merged)


(197, 101, 60)
testing units:  [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196] [ 6.0228677 13.394905  11.293156  10.928486   8.945223   3.7195044
 14.2912655  9.407461   3.8813028 23.122099  13.687571  11.72106
  3.6515245  5.1513267  7.724716   5.

In [16]:
# ptps = templates_yass.ptp(1).max(1)
# idx = np.argsort(ptps)[::-1]

# idx_in = idx#[:3]

In [19]:
data = np.fromfile('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/data_correct_order.bin','int16')
n_chans = 60
data_2D = data.reshape(-1,n_chans)
print (data_2D.shape)

(19200000, 60)


In [20]:
templates = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output/templates.npy')
print (templates.shape)
max_chans= templates.ptp(1).argmax(1)
ptps = templates.ptp(1).max(1)

(217, 101, 60)


In [21]:
# reloat template 



#fname = '/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/standardized.bin'
#data_type = 'float32'
fname = '/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/data_correct_order.bin'
data_type = 'int16'

n_channels = 60
n_times = 151

#spike_train = np.load('/media/cat/1TB/data/synthetic/run3/ground_truth/spike_train_ground_truth.npy')
#spike_train_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output/spike_train.npy')
spike_train_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output_original/spike_train.npy')
spike_train_gt = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/ground_truth/spike_train.npy')
    
unit_yass = 199

# load yass waveforms
idx = np.where(spike_train_yass[:,1]==unit_yass)[0]
spikes = np.int32(spike_train_yass[idx,0])-n_times//2
wfs_yass = binary_reader_waveforms(fname, n_channels, n_times, spikes, data_type)
print (wfs_yass.shape)

template_yass = np.mean(wfs_yass,0)
print (template_yass.shape)

(2873, 151, 60)
(151, 60)


In [22]:
print (template_yass.ptp(0).max(0))
plt.plot(template_yass)

3106.1921336581972


In [25]:
# COMPUTE SNR OF EACH CHANNEL;

max_chan = max_chans[unit]

#voltage_snippit = data[max_chan]

mad = np.median(data_2D[max_chan]-np.mean(data_2D[max_chan],0))
print ("Mad: ", mad)

sigma = mad/0.6745
print ("sigma: ", sigma)

SNR = np.max(np.abs(template_yass[:,max_chan]))/sigma

print ("SNR: ", SNR, " ptp: ", ptps[unit])



Mad:  6.866666666666667
sigma:  10.180380528786756
SNR:  87.62549783048739  ptp:  12.1686535


In [3]:
results_merged = np.load('/media/cat/4TBSSD/data/spikingforest/results_merged.npy')
results = np.load('/media/cat/4TBSSD/data/spikingforest/results_original.npy')

spike_train_gt = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/ground_truth/spike_train.npy')
templates_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output_original/templates.npy', allow_pickle=True)
spike_train_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output_original/spike_train.npy')

# plot original results
fig =plt.figure()
#plot_results(templates_yass, spike_train_yass, spike_train_gt, results, np.arange(idx.shape[0]))

# plot merged results
#fig =plt.figure()
spike_train_yass_merged = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output/spike_train_merged.npy')
templates_merged = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output/templates_merged.npy')
idx_in = np.arange(templates_merged.shape[0])
print ("res", results_merged.shape)
plot_results(templates_merged, spike_train_yass_merged, spike_train_gt, results_merged, idx_in)


res (197, 8)
(197, 101, 60)
len results:  197
yass:  38 , gt:  480.0  ptp:  52.407104  accuracy:  1.0
yass:  48 , gt:  459.0  ptp:  35.12999  accuracy:  0.9881450488145049
yass:  51 , gt:  390.0  ptp:  33.673965  accuracy:  0.9917330971361087
yass:  54 , gt:  502.0  ptp:  55.53207  accuracy:  0.9988439306358381
yass:  70 , gt:  570.0  ptp:  29.937677  accuracy:  0.9916559691912709
yass:  86 , gt:  516.0  ptp:  68.72615  accuracy:  1.0
yass:  91 , gt:  235.0  ptp:  33.367317  accuracy:  0.9928453494771602
yass:  127 , gt:  519.0  ptp:  30.043097  accuracy:  0.9857716161984678
yass:  134 , gt:  557.0  ptp:  34.38801  accuracy:  0.9680534918276374
yass:  143 , gt:  385.0  ptp:  35.589123  accuracy:  0.9979674796747967
yass:  149 , gt:  561.0  ptp:  62.575676  accuracy:  0.9980601357904947
yass:  152 , gt:  540.0  ptp:  35.968914  accuracy:  0.9947780678851175
yass:  178 , gt:  567.0  ptp:  35.639107  accuracy:  0.9764749813293503
yass:  179 , gt:  541.0  ptp:  49.622517  accuracy:  0.9996

In [10]:
idx = np.where(np.logical_and(ptps>28.5, ptps<29.5))
print (idx)
selected_unit = idx[0]

print (len(results))

print (results[selected_unit])

(array([199]),)
221
[[2.17000000e+02 9.70414201e-01 5.04615385e-01 2.65200000e+03
  8.50000000e+01 2.78800000e+03 2.87300000e+03 5.44000000e+03]]


In [74]:
# find the split off match between two units
for k in range(len(results)):
    if results[k][0]==217:
        print (k)


52
199


In [25]:
print (templates_yass.shape)
temp1 = templates_yass[52]
temp2 = templates_yass[199]
ax1 = plt.subplot(2,2,1)

plt.plot(temp1,c='black')
plt.plot(temp2, c='red')
plt.title("Raw templates")

temp1_norm = -temp1/np.min(temp1)
temp2_norm = -temp2/np.min(temp2)
ax1 = plt.subplot(2,2,2)
plt.plot(temp1_norm, c='black')
plt.plot(temp2_norm, c='red')
plt.title("Normalized templates")

ax3 = plt.subplot(2,2,3)
diff = temp1_norm - temp2_norm
plt.plot(diff,c='green')
plt.ylim(-.1,.1)
plt.title("Diff between normalized templates")
plt.ylabel("Normalized value")
plt.suptitle("Bionet simulation; analysis of bursting unit 217; \nyass oversplit into units 52 (black) and 199 (red)")

ax3 = plt.subplot(2,2,4)
diff = (temp1_norm - temp2_norm)* np.min(temp1)
plt.plot(diff,c='green')
#plt.ylim(-.1,.1)
plt.title("Diff between normalized templates")
plt.ylabel("SU values")
plt.suptitle("Bionet simulation; analysis of bursting unit 217; \nyass oversplit into units 52 (black) and 199 (red)")



plt.show()

(221, 101, 60)


In [40]:
#     return (best_match_test, purity, accuracy,
#            n_fn, n_fp, tot_matches, tot_sorted, tot_ground_truth)

# thresholded purity metric
results_merged = np.load('/media/cat/4TBSSD/data/spikingforest/results_merged.npy')

print (results_merged.shape)
accuracy = results_merged[:,2]
sizes = results_merged[:,6]
clr_out = 'red'
#clr_out = 'blue'
data=np.array(accuracy)


ax=plt.subplot(1,1,1)
plt.scatter(ptps, data, s=sizes/20.,c=clr_out,alpha=.5)
#ax=plt.subplot(1,2,2)
idx = np.argsort(ptps)[::-1]


cum_purity = []
for k in range(idx.shape[0]):
    cum_purity.append(data[idx][:k].mean(0))
    
plt.plot(ptps[idx],cum_purity,linewidth=6, c='black')
plt.ylim(0,1)
plt.plot([8,8],[0,1],'r--',linewidth=3,c='black')
plt.title("accuracy")
plt.show()


(197, 8)


/home/cat/anaconda3/envs/spikingforest/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: Mean of empty slice.
/home/cat/anaconda3/envs/spikingforest/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [27]:
unit = 100
plt.plot(templates_yass[unit])
plt.title("Unit: "+str(unit))
plt.show()

In [21]:
# units=np.arange(5)
# units = [good_units[3]]

# for unit in units:
#     fig = plt.figure()
#     ax = plt.subplot(131)
#     #plt.plot(templates[unit].T)
#     plt.plot(wfs_array[unit].mean(0))

#     ax = plt.subplot(132)
#     #spikes = spikes[idx]
#     spikes = times[unit]
#     print ("spikes:", spikes.shape, "ptps: ", ptps[unit].shape)
#     plt.scatter(spikes/30000., ptps[unit])

#     ax=plt.subplot(133)
#     max_chan = wfs_array[unit].mean(0).ptp(0).argmax(0)#templates[unit].ptp(0).argmax(0)
#     print (wfs_array[unit].shape)

#     cmap = cm.get_cmap('viridis',wfs_array[unit].shape[0])
#     clrs = cmap(np.arange(wfs_array[unit].shape[0]))
#     print (clrs)
#     for k in range(0,wfs_array[unit].shape[0],10):
#         plt.plot(wfs_array[unit][k].T,c=cmap(k))


# plt.show()



spikes: (8210,) ptps:  (8210,)
(8210, 101)
[[0.267004 0.004874 0.329415 1.      ]
 [0.267004 0.004874 0.329415 1.      ]
 [0.267004 0.004874 0.329415 1.      ]
 ...
 [0.993248 0.906157 0.143936 1.      ]
 [0.993248 0.906157 0.143936 1.      ]
 [0.993248 0.906157 0.143936 1.      ]]


In [11]:



#fname = '/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/standardized.bin'
#data_type = 'float32'
fname = '/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/data_correct_order.bin'
data_type = 'int16'

n_channels = 60
n_times = 151

#spike_train = np.load('/media/cat/1TB/data/synthetic/run3/ground_truth/spike_train_ground_truth.npy')
#spike_train_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output/spike_train.npy')
spike_train_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output_original/spike_train.npy')
spike_train_gt = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/ground_truth/spike_train.npy')
    
    
#units = [199]
templates = []
wfs_array = []

# yass:  19 , gt:  561  ptp:  62.675797
# yass:  25 , gt:  235  ptp:  48.744278
# yass:  100 , gt:  541  ptp:  49.390488
# yass:  138 , gt:  516  ptp:  68.41851
# yass:  165 , gt:  480  ptp:  52.412918
# yass:  187 , gt:  502  ptp:  55.726055
            
unit_yass = 199
unit_gt = 217+1

# load yass waveforms
idx = np.where(spike_train_yass[:,1]==unit_yass)[0]
spikes = np.int32(spike_train_yass[idx,0])-n_times//2
wfs_yass = binary_reader_waveforms(fname, n_channels, n_times, spikes, data_type)
print (wfs_yass.shape)

# load 
idx = np.where(spike_train_gt[:,1]==unit_gt)[0]
spikes = np.int32(spike_train_gt[idx,0])-n_times//2+19
wfs_gt = binary_reader_waveforms(fname, n_channels, n_times, spikes, data_type)
print (wfs_gt.shape)    

(2873, 151, 60)
(5544, 151, 60)


In [65]:
template_yass = wfs_yass.mean(0)
print (template_yass.shape)
max_chans = template_yass.ptp(0)
idx = np.argsort(max_chans)[::-1]
print (idx)

for k in range(10):
    ax=plt.subplot(2,5,k+1)
    print (idx[k],wfs_yass.mean(0).shape)
    plt.plot(wfs_yass.mean(0)[:,idx[k]],c='red')
    plt.plot(wfs_gt.mean(0)[:,idx[k]],c='black')
    plt.xticks([])
    plt.yticks([])
    
plt.suptitle("Red: yass;  black: ground truth")
plt.show()

(151, 60)
[32 30 34 33 31 35 36 28 29 37 26 27 38 22 24 23 25 39 20 21 19 18 40 17
 16 41 14 15 12 42 13 43 10 11 44  8  9 45  6  7 46 47  4  5 48 49  2  3
  0  1 50 51 52 53 55 54 57 56 58 59]
32 (151, 60)
30 (151, 60)
34 (151, 60)
33 (151, 60)
31 (151, 60)
35 (151, 60)
36 (151, 60)
28 (151, 60)
29 (151, 60)
37 (151, 60)


In [69]:
max_chan = wfs_yass.mean(0).ptp(0).argmax(0)

plt.plot(wfs_yass[:,:,max_chan].T,c='red', alpha=.01)
plt.plot(wfs_gt[:,:,max_chan].T,c='black', alpha=.01)
plt.show()

In [75]:
max_chan = templates[0].ptp(0).argmax(0)
plt.plot(wfs[:,:,max_chan].T, c= 'black', alpha=.1)
plt.show()

In [38]:
plt.plot(templates[0])
plt.show()

In [42]:
data = np.fromfile('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/data.bin','int16')
print (data.shape)

(1152000000,)


In [44]:
data2D = data.reshape(60,-1).T
for k in range(60):
    plt.plot(data2D[:10000, k]+k*100,c='black')
plt.show()

In [45]:
data2D.ravel().tofile('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/data_correct_order.bin')

In [7]:
spike_train_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output/spike_train.npy')
print (spike_train_yass.shape)

(1081156, 2)


In [72]:
import csv

with open('employee_birthday.txt') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            print(f'\t{row[0]} works in the {row[1]} department, and was born in {row[2]}.')
            line_count += 1
    print(f'Processed {line_count} lines.')
    
print (data)